# Finding the road lanes using OpenCV

Description

First, as images tend to be distorted, I will use the OpenCV library to calibrate and undistort the input image

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob # To import all the calibration images

In [2]:
def calibrate(objpoints,imgpoints):

    objp = np.zeros((6*9,3), np.float32) 
    objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

    # Make a list of calibration images
    images = glob.glob('/Users/abylikhsanov1/AI/carnd/term1/advanced-lane-lines/camera_cal/calibration*.jpg') # Please update if needed
    for fname in images:
        image = cv2.imread(fname)
        gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (9,6), None) # Finding any distorted pixel locations
        if ret is True: # If we found any, we shall append the coordinates and the pixels itself
            objpoints.append(objp)
            imgpoints.append(corners)
            cv2.drawChessboardCorners(image, (9,6), corners, ret)
    cv2.destroyAllWindows()

Now, as we finished to get the distorted points from the sample images, we know our image points (pixels) which need to be undistorted and the constant object points (coordinates) as our final goal location.

In [50]:
def process_image(img):
    %matplotlib inline
    #img = cv2.imread('test_images/straight_lines1.jpg') # Now, let's undistort our first road image
    #img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    fig,(ax1) = plt.subplots(1,1,figsize=(10,5))
    ax1.imshow(img)
    # Calibrating and removing the distortion
    img_size = (img.shape[1],img.shape[0]) # We are making img_size[0] as x axis values and [1] as y axis
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints,img_size,None,None) # Calibration, moving imgpoints towards the coordinate
    dst = cv2.undistort(img, mtx, dist, None, mtx)

    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=30)
    ax2.imshow(dst)
    ax2.set_title('Undistorted Image', fontsize=30)
    global count
    global left_fit
    global right_fit
    combined_binary = gradient(dst)
    warped,M = warp(combined_binary)
    if(count == 0):
        result,left_fitx,right_fitx,ploty,left_fit,right_fit = fit1(warped)
    elif(count > 0):
        result,left_fitx,right_fitx,ploty = fit2(warped,left_fit,right_fit)
    final = unwarp(result,img,left_fitx,right_fitx,ploty,M)
    count = count + 1
    return final

After our image undistortion, the next step will be to detect the gradients of the image using the Sobel operator. We will actually create 3 methods and choose one of them toidentify pixels where the gradient of an image falls within a specified threshold range.

In [4]:
# Full code, consists of 3 methods, the Sobel (x or y), the magniture (x and y combined) and the direction (the angle of the gradient)

def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculates the directional gradient
    # Apply threshold
    if orient == 'x':
        sobel = cv2.Sobel(img,cv2.CV_64F,1,0)
    elif orient == 'y':
        sobel = cv2.Sobel(img,cv2.CV_64F,0,1)
    sobel_abs = np.absolute(sobel)
    sobel_bin = np.uint8((255*sobel_abs/np.max(sobel_abs)))
    grad_binary = np.zeros_like(sobel_bin)
    grad_binary[(sobel_bin>=thresh[0])&(sobel_bin<=thresh[1])] = 1
    return grad_binary

def mag_thresh(img, sobel_kernel=3, thresh=(0, 255)):
    # Calculates the gradient magnitude
    # Apply threshold
    sobelx = cv2.Sobel(img,cv2.CV_64F,1,0)
    sobely = cv2.Sobel(img,cv2.CV_64F,0,1)
    sobel_abs = np.absolute(sobelx+sobely)
    sobel_bin = np.uint8((255*sobel_abs/np.max(sobel_abs)))
    mag_binary = np.zeros_like(sobel_bin)
    mag_binary[(sobel_bin>=thresh[0])&(sobel_bin<=thresh[1])] = 1
    return mag_binary

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculates the gradient direction
    # Apply threshold
    sobelx = np.absolute(cv2.Sobel(img,cv2.CV_64F,1,0))
    sobely = np.absolute(cv2.Sobel(img,cv2.CV_64F,0,1))
    sobel_arctan = np.uint(np.arctan(sobely,sobelx))
    dir_binary = np.zeros_like(sobel_arctan)
    dir_binary[(sobel_arctan>=thresh[0])&(sobel_arctan<=thresh[1])] = 1
    return dir_binary

k_size = 7 # To make our image smoother, I have concluded, that this Kernel size is optimal.

#### IGNORE THE BELOW CODE IN THIS CELL, IT IS FOR THE TEST MODE ONLY ####

#gradx = abs_sobel_thresh(gray, orient='x', sobel_kernel=k_size, thresh=(10, 240))
#grady = abs_sobel_thresh(gray, orient='y', sobel_kernel=k_size, thresh=(30, 100))
#mag_binary = mag_thresh(gray, sobel_kernel=k_size, mag_thresh=(30, 100))
#dir_binary = dir_threshold(gray, sobel_kernel=k_size, thresh=(0.3, 1.3))

#combined = np.zeros_like(dir_binary)
#combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1

#plt.imshow(combined,cmap='gray')
#plt.show()

#hls = cv2.cvtColor(dst,cv2.COLOR_RGB2HLS)
#f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,10))
#ax1.set_title('Image in grayscale, H space')
#ax1.imshow(hls[:,:,0], cmap='gray')
#ax2.set_title('Image in grayscale, L space')
#ax2.imshow(hls[:,:,1], cmap='gray')
#ax3.set_title('Image in grayscale, S space')
#ax3.imshow(hls[:,:,2], cmap='gray')

Now, that we have defined our methods, we can experiment, which of them will work best with our image. We want to identify the line lanes. More over, in the next code cell, I will transform the image to HLS space, as you will see next, the "Saturation" value will make the non-white lanes more enhanced and actually performs good on shadows.

In [5]:
def gradient(dst):
    ## 1
    hls = cv2.cvtColor(dst,cv2.COLOR_RGB2HLS)
    gray = cv2.cvtColor(dst,cv2.COLOR_RGB2GRAY)
    luv = cv2.cvtColor(dst,cv2.COLOR_RGB2LUV)
    r = (230,255)
    g = (230,255)
    s = (10,200)
    l = (1,120)
    #r = (240,255)
    #g = (240,255)
    #s = (30,255)
    #l = (30,200)

    r_mag = abs_sobel_thresh(dst[:,:,0],sobel_kernel=3,thresh=r)
    g_mag = abs_sobel_thresh(dst[:,:,1],sobel_kernel=3,thresh=g)
    s_mag = abs_sobel_thresh(hls[:,:,2],sobel_kernel=9,thresh=s)
    l_mag = mag_thresh(luv[:,:,0],sobel_kernel=9,thresh=l)
    combined_binary = np.zeros_like(r_mag)
    combined_binary[((r_mag==1) & (g_mag==1)) | (s_mag==1)] = 1
    
    if len(combined_binary.shape) > 2:
        channel_count = combined_binary.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    vertices = np.int32([[[190,700],[560,450],[800,450],[1200,700]]])
    mask = np.zeros_like(combined_binary)
    cv2.fillPoly(mask, vertices,ignore_mask_color)
    combined_binary = cv2.bitwise_and(combined_binary, mask)
    
    
    ## 2
    #s_mag = abs_sobel_thresh(hls[:,:,2],sobel_kernel=17,thresh=(40, 200))
    #l_mag = abs_sobel_thresh(hls[:,:,1],orient='x',sobel_kernel=17,thresh=(120, 150))
    #gray_mag = abs_sobel_thresh(hls[:,:,0],orient='x',sobel_kernel=17,thresh=(200,230))
    
    #combined_binary = np.zeros_like(gray_mag)
    #combined_binary[(s_mag==1) | (l_mag==1) | (gray_mag==1)] = 1
    #gray = cv2.cvtColor(dst,cv2.COLOR_RGB2GRAY)
    #hls = cv2.cvtColor(dst,cv2.COLOR_RGB2HLS)

    
    
    
    # Sobel X on gray:
    #sobel_gray = abs_sobel_thresh(gray, orient='x', sobel_kernel=k_size, thresh=(20, 200))

    # Sobel X on S (HLS):
    #sobel_s = abs_sobel_thresh(hls[:,:,2], orient='x', sobel_kernel=k_size, thresh=(180, 255))

    # Combine the two binary thresholds
    #combined_binary = np.zeros_like(sobel_gray)
    #combined_binary[(sobel_s == 1) | (sobel_gray == 1)] = 1

    # Plotting thresholded images
    fig,(ax1,ax2,ax3,ax4) = plt.subplots(1,4,figsize=(20,10))
    ax1.imshow(dst[:,:,0])
    ax2.imshow(dst[:,:,1])
    ax3.imshow(hls[:,:,2])
    ax4.imshow(luv[:,:,0])

    return combined_binary

Now, having a very good and enhanced lane lines, I will warp the image, in order to visualise the lanes in the right perspective (left and right lines have to be parallel). After warping the image (viewing from the top), the next task will be to construct the polynomial based on the line lanes.

In [24]:
# Time to warp the image
def warp(combined_binary):
    img_size = (combined_binary.shape[1],combined_binary.shape[0])
    #src = np.float32([[207,720],[570,468],[714,468],[1106,720]])
    bottom_left = [320,720] 
    bottom_right = [980, 720]
    top_left = [320, 0]
    top_right = [980, 0]
    src = np.float32([[261,720],[559,475],[713,475],[1200,720]])
    #np.float32([[261,720],[559,475],[713,475],[1200,720]])
    #np.float32([[261,720],[559,475],[713,475],[1200,720]])
    #np.float32([[256,688],[544,492],[752,492],[1044,688]])
    dest = np.float32([bottom_left,top_left,top_right,bottom_right])
    M = cv2.getPerspectiveTransform(src,dest)
    warped = cv2.warpPerspective(combined_binary,M,img_size)
# Plotting warped and non images
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.set_title('Combined S channel and gradient thresholds')
    ax1.imshow(combined_binary,cmap='gray')

    ax2.set_title('Warped image')
    ax2.imshow(warped, cmap='gray')
    return warped,M

In order to construct the polynomial across the lane lines, I will start by finding the most dense x axis pixel values. The function below explains how the x axis points are found and based on that, the best polynomial fit is being drawn, which effectively are the lane lines.

In [51]:
def fit1(warped):
    # Viewing the most dense pixel region (sum of 1 on the X scale)
    histogram = np.sum(warped[int(warped.shape[0]/2):,:],axis=0)
    #plt.plot(histogram)
    out_img = warped

    #In order to fit the polynomial for the lines, I will divide the image by 2 on x axis, to seperate left and right lanes
    midpoint = int(histogram.shape[0]/2)
    left_side = np.argmax(histogram[:midpoint]) # Getting the most dense pixel region at x axis, argmax returns the index
    right_side = np.argmax(histogram[midpoint:]) + midpoint # Getting the most dense pixel region at x axis right side
# As the maximum Y value is 720, I will choose to divide it to 9 windows
    windows = 9

# Set height of windows
    window_height = np.int(warped.shape[0]/windows) # In this code, this is int size of 80 (80 pixels)

# Identify the x and y positions of all nonzero pixels in the image
    nonzero = warped.nonzero() # Pixel locations where pixel is 1, [1] = x, [0] = y
    nonzeroy = np.array(nonzero[0]) 
    nonzerox = np.array(nonzero[1])

# Current positions to be updated for each window
    leftx_current = left_side 
    rightx_current = right_side

# Set the width of the windows +/- margin
    margin = 100
# Set minimum number of pixels found to recenter window
    minpix = 50
# Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0])# All Y values
    

# Step through the windows one by one (Search only for the first frame)
    for window in range(windows): # Looping in 9 steps
    # Identify window boundaries in x and y (and right and left)
        win_y_low = warped.shape[0] - (window+1)*window_height # Loop 1, 0+1 * 80 = 80 px
        win_y_high = warped.shape[0] - window*window_height # Loop 1, 0 px, this is a top value, as y values are from the top to the bottom
        win_xleft_low = left_side - margin # Setting the square boundaries, from the current found lane piece
        win_xleft_high = left_side + margin
        win_xright_low = right_side - margin
        win_xright_high = right_side + margin
    # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 2) 
    # Identify the nonzero pixels in x and y within the window
        left_nzero_values = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
         (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]  # Getting the pixel locations, where pixel>1
        right_nzero_values = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
    # Append these indices to the lists
        left_lane_inds.append(left_nzero_values) # Left_lane_inds is the list of pixel locations in that margin box
        right_lane_inds.append(right_nzero_values)
    # If you found > minpix pixels, recenter next window on their mean position
        if len(left_nzero_values) > minpix:
            leftx_current = np.int(np.mean(nonzerox[left_nzero_values])) # Enhance: Try to get the argmax of np.sum of the location
        if right_nzero_values.size > minpix:        
            rightx_current = np.int(np.mean(nonzerox[right_nzero_values])) # For all the x values with pixels>1, we get the mean of that


# Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
# Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds] # Get the
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
# Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    out_img = np.dstack((warped, warped, warped))*255
    window_img = np.zeros_like(out_img) # Getting the blank image to display the curves
# Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] ) # All the y values
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

# Generate a polygon to illustrate the search window area
# And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+(right_fitx-left_fitx),ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))

# Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))

    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    return result,left_fitx,right_fitx,ploty,left_fit,right_fit
        

In [32]:
def fit2(warped,left_fit,right_fit):
    margin = 100
# Set minimum number of pixels found to recenter window
    minpix = 50
# Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0])# All Y values
    out_img = warped
    nonzero = warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
    left_fit[1]*nonzeroy + left_fit[2] + margin))) 

    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
    right_fit[1]*nonzeroy + right_fit[2] + margin)))  

# Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
# Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
# Generate x and y values for plotting
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        
    out_img = np.dstack((warped, warped, warped))*255
    window_img = np.zeros_like(out_img) # Getting the blank image to display the curves
# Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

# Generate a polygon to illustrate the search window area
# And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+(right_fitx-left_fitx),ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))

# Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))

    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    return result,left_fitx,right_fitx,ploty

After finding the polynomials, the next step is to unwarp them into the original image.

In [26]:
def unwarp(result,image,left_fitx,right_fitx,ploty,M):
    # Create an image to draw the lines on
    #warp_zero = np.zeros_like(result).astype(np.uint8)
    #color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    warp_zero = np.zeros_like(image)
    #color_warp = np.dstack((result, result, result))*255

# Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    bottom_left = [left_fitx[0],result.shape[0]] 
    bottom_right = [right_fitx[0], result.shape[0]]
    top_left = [left_fitx[-1], 0]
    top_right = [right_fitx[-1], 0]
    #pts = np.array([bottom_left,bottom_right,top_left,top_right])

# Draw the lane onto the warped blank image
    cv2.fillPoly(warp_zero, np.int_([pts]), (0,255,0))
    src = np.float32([bottom_left,top_left,top_right,bottom_right])
    dest = np.float32([[261,720],[559,475],[713,475],[1200,720]])
    #np.float32([[261,720],[559,475],[713,475],[1200,720]])
    #np.float32([[261,720],[559,475],[713,475],[1200,720]])
    #np.float32([[256,688],[544,492],[752,492],[1044,688]])
    M_inv = cv2.getPerspectiveTransform(src,dest)

# Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(warp_zero, M_inv, (image.shape[1], image.shape[0])) 
# Combine the result with the original image

    final = cv2.addWeighted(image, 1, newwarp, 0.3, 0)
    fig,(ax1) = plt.subplots(1,1,figsize=(15,5))
    ax1.imshow(final)
    
    
    #bottom_left = [left_fitx[0],result.shape[0]] 
    #bottom_right = [right_fitx[0], result.shape[0]]
    #top_left = [left_fitx[-1], 0]
    #top_right = [right_fitx[-1], 0]
    #dest = np.float32([[207,720],[570,468],[714,468],[1106,720]])
    ##dest = np.float32([[256,688],[544,492],[752,492],[1044,688]])
    #src = np.float32([bottom_left,top_left,top_right,bottom_right])
    #img_size = (img.shape[1],img.shape[0])
    #M = cv2.getPerspectiveTransform(src,dest)
    #result_unwarp = cv2.warpPerspective(result,M,img_size)
# Plotting warped and non images
    #if len(img.shape) > 2:
     #   channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
      #  ignore_mask_color = (255,) * channel_count
    #else:
     #   ignore_mask_color = 255

    #mask = np.zeros_like(result)
    #vertices = np.array([[[256,688],[544,492],[752,492],[1100,688]]],dtype=np.int32)
    #cv2.fillPoly(mask, vertices,ignore_mask_color)
    ###returning the image only where mask pixels are nonzero
    #masked_image = cv2.bitwise_and(result_unwarp, mask)
    #final = cv2.addWeighted(img, 1, masked_image, 1, 1)
    #f, (ax1) = plt.subplots(1, 1, figsize=(20,10))

    #ax1.set_title('Warped image')
    #ax1.imshow(final)
    return final
    #cv2.imwrite( "output_images/straight_lines1.jpg", cv2.cvtColor(final,cv2.COLOR_BGR2RGB));

In [9]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
white_output = 'output_videos/project_video.mp4'
clip1 = VideoFileClip("project_video.mp4").subclip(0,1)

    # Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.
count = 0
left_fit = np.array([0,0,0], dtype='float')
right_fit = np.array([0,0,0], dtype='float')
calibrate(objpoints,imgpoints)
white_clip = clip1.fl_image(process_image) 
%time white_clip.write_videofile(white_output, audio=False)
print("The number is: ",count)

[MoviePy] >>>> Building video output_videos/project_video.mp4
[MoviePy] Writing video output_videos/project_video.mp4



 62%|██████▏   | 16/26 [00:19<00:11,  1.17s/it]

In [45]:
class Line():
    def __init__(self,first=True,recent_xfitted=[],bestx=None,best_fit=None,current_fit=[np.array([False])],
                 radius_of_curvature = None,
                 line_base_pos = None,diffs = np.array([0,0,0], dtype='float'),allx = None,ally=None):
        # was the line detected in the last iteration?
        self.first = first
        # x values of the last n fits of the line
        self.recent_xfitted = recent_xfitted
        #average x values of the fitted line over the last n iterations
        self.bestx = bestx
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = best_fit
        #polynomial coefficients for the most recent fit
        self.current_fit = current_fit
        #radius of curvature of the line in some units
        self.radius_of_curvature = radius_of_curvature
        #distance in meters of vehicle center from the line
        self.line_base_pos = line_base_pos
        #difference in fit coefficients between last and new fits
        self.diffs = diffs
        #x values for detected line pixels
        self.allx = allx
        #y values for detected line pixels
        self.ally = ally